In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [6]:
df = pd.read_csv('https://github.com/susanli2016/NLP-with-Python/raw/master/data/ner_dataset.csv', encoding = "ISO-8859-1")
df = df[:50000]
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [7]:
df = df.fillna(method='ffill')

/tmp/ipykernel_10492/569672361.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


A seguir podemos ver a quantidade de setenças, palavras e tags únicas.

In [8]:
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

(2270, 7464, 17)

Assim como a distribuição das tags:

In [9]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-art,48
1,B-eve,39
2,B-geo,1490
3,B-gpe,968
4,B-nat,18
5,B-org,959
6,B-per,789
7,B-tim,880
8,I-art,27
9,I-eve,33


In [10]:
X = df.drop('Tag', axis=1)
X.head()

,Sentence #,Word,POS
0,Sentence: 1,Thousands,NNS
1,Sentence: 1,of,IN
2,Sentence: 1,demonstrators,NNS
3,Sentence: 1,have,VBP
4,Sentence: 1,marched,VBN


In [11]:
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
X.shape

(50000, 9774)

In [12]:
y = df.Tag.values

In [13]:
classes = np.unique(y)

In [14]:
classes = classes.tolist()
classes

['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim',
 'O']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state=0)

### Multi-Layer Perceptron

In [16]:
new_classes = classes.copy()
new_classes.pop()

'O'

In [17]:
per = MLPClassifier(verbose=10, max_iter=10)
per.fit(X_train, y_train)

Iteration 1, loss = 1.15990192
Iteration 2, loss = 0.32239219
Iteration 3, loss = 0.22462732
Iteration 4, loss = 0.15981564
Iteration 5, loss = 0.11775649
Iteration 6, loss = 0.09001785
Iteration 7, loss = 0.07120744
Iteration 8, loss = 0.05761456
Iteration 9, loss = 0.04752472
Iteration 10, loss = 0.03961854


/home/lucasqueiros/Downloads/yes/envs/nlp/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(max_iter=10, verbose=10)

In [18]:
print(classification_report(y_pred=per.predict(X_test), y_true=y_test, labels=new_classes))

/home/lucasqueiros/Downloads/yes/envs/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lucasqueiros/Downloads/yes/envs/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       B-art       0.67      0.24      0.35        17
       B-eve       0.33      0.09      0.14        11
       B-geo       0.66      0.72      0.69       441
       B-gpe       0.86      0.77      0.81       297
       B-nat       0.00      0.00      0.00         3
       B-org       0.54      0.53      0.53       272
       B-per       0.56      0.57      0.56       233
       B-tim       0.81      0.80      0.81       246
       I-art       0.50      0.25      0.33         4
       I-eve       1.00      0.18      0.31        11
       I-geo       0.67      0.44      0.53       107
       I-gpe       0.00      0.00      0.00        10
       I-nat       0.00      0.00      0.00         2
       I-org       0.55      0.53      0.54       180
       I-per       0.63      0.63      0.63       279
       I-tim       0.45      0.16      0.23        83

   micro avg       0.66      0.62      0.64      2196
   macro avg       0.51   

/home/lucasqueiros/Downloads/yes/envs/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Regressão Logística

In [20]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [21]:
print(classification_report(y_pred=lr.predict(X_test), y_true=y_test, labels=new_classes))

/home/lucasqueiros/Downloads/yes/envs/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       B-art       1.00      0.06      0.11        17
       B-eve       0.00      0.00      0.00        11
       B-geo       0.56      0.85      0.68       441
       B-gpe       0.96      0.58      0.72       297
       B-nat       0.00      0.00      0.00         3
       B-org       0.57      0.51      0.54       272
       B-per       0.61      0.49      0.55       233
       B-tim       0.90      0.77      0.83       246
       I-art       0.00      0.00      0.00         4
       I-eve       1.00      0.09      0.17        11
       I-geo       0.82      0.30      0.44       107
       I-gpe       0.00      0.00      0.00        10
       I-nat       0.00      0.00      0.00         2
       I-org       0.60      0.47      0.53       180
       I-per       0.57      0.70      0.63       279
       I-tim       1.00      0.06      0.11        83

   micro avg       0.65      0.60      0.62      2196
   macro avg       0.54   

/home/lucasqueiros/Downloads/yes/envs/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lucasqueiros/Downloads/yes/envs/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Naive Bayes

In [22]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB()

In [23]:
print(classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels = new_classes))

/home/lucasqueiros/Downloads/yes/envs/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00        17
       B-eve       0.00      0.00      0.00        11
       B-geo       0.46      0.92      0.61       441
       B-gpe       0.94      0.56      0.70       297
       B-nat       0.00      0.00      0.00         3
       B-org       0.51      0.47      0.49       272
       B-per       0.68      0.42      0.52       233
       B-tim       0.89      0.73      0.80       246
       I-art       0.00      0.00      0.00         4
       I-eve       0.00      0.00      0.00        11
       I-geo       1.00      0.08      0.16       107
       I-gpe       0.00      0.00      0.00        10
       I-nat       0.00      0.00      0.00         2
       I-org       0.65      0.34      0.45       180
       I-per       0.60      0.59      0.60       279
       I-tim       0.00      0.00      0.00        83

   micro avg       0.60      0.55      0.57      2196
   macro avg       0.36   

/home/lucasqueiros/Downloads/yes/envs/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lucasqueiros/Downloads/yes/envs/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
